In [39]:
import pandas as pd
import numpy as np
import csv
import os

In [40]:
abs_path = 'ecosys/ecosys_initial_configurations/'

locales = ['cool_temp_maize_soybean', 'warm_temp_maize_soybean_irrigated', 'warm_temp_maize-soybean_dryland']

local = locales[0]

cfg_files_pth = abs_path + local

out_files = ['soil_c','surf_water','soil_temp','flux_soc','soil_water','n_flux','p_flux','temp']

years = 8
growth_strt = 2015

In [41]:
mdays=[31,28,31,30,31,30,31,31,30,31,30,31]
def is_leap(year):
    leap=False
    if year % 4 == 0 and year % 100 != 0:
        leap=True
    elif year % 100 == 0:
        leap=False
    elif year % 400 ==0:
        leap=True
    else:
        leap=False
    return leap
def dayofyear(year,month,day):
    doy=0
    for mm in range(0,month):
        days=mdays[mm]
        if mm==1:
            if isleap(year):
                days=29
        for dd in range(0,days):
            if mm < month-1:
                doy=doy+1
            else:
                if dd < day:
                    doy=doy+1
    return doy

In [14]:
def handle_file(line):
    strps = line.strip().split()
    
    for strp in strps:
        print(strp)
        if line[0].isalpha() and strp not in out_files and strp != 'NO' and 'weather' not in strp:
            if strp in plant_f.keys():
                strp = plant_f[strp]

            with open(pth + '/' + strp, 'r') as in_file, open(save_path + '/' + strp  + '.csv', 'w', newline='', encoding='utf-8') as out_file:
                    writer = csv.DictWriter(out_file, fieldnames =['0'])
                    writer.writeheader()
                    for line in in_file:
                        sep = ' '
                        if ',' in line:
                            sep = ','

                        line = [elem.strip() for elem in line.split(sep)]
                        line = [elem for elem in line if elem]

                        # Write elements to output file
                        for elem in line:
                            if os.path.isfile(pth + '/' + elem):
                                handle_file(elem)
                                continue

                            writer.writerow({'0': elem})
                        
def init_cfg_to_csv(file):
    with open(file, 'r') as rnfl:
        for line in rnfl:
            handle_file(line) 
            
def join_planting_data(pth, file):
    fields = []
    with open(os.path.join(pth , file), 'r') as rnfl:
        for line in rnfl:
            strps = line.strip().split()
            
            for strp in strps:
                if strp[0].isalpha():
                    fields.append(strp)
                    
    with open(save_path + '/' + file  + '.csv', 'w', newline='', encoding='utf-8') as out_file:
        writer = csv.DictWriter(out_file, fieldnames = fields)
        writer.writeheader()
        for field in fields: 
            
            to_open = field
            if to_open in plant_f.keys():
                to_open = plant_f[to_open]


            with open(pth + '/' + to_open, 'r') as in_file:
                for val in in_file:
                    sep = ' '
                    if ',' in val:
                        sep = ','
                        
                    line = [elem.strip() for elem in val.split(sep) if elem.strip()]
                    
                    for elem in line:
                        writer.writerow({field: elem})


In [42]:
def handle_soil(df: pd.DataFrame, soil_path: str)-> pd.DataFrame:
    soil_prfl_vars = {
    "initial bulk density (Mg m-3,0=water) ": 2,
    "sand contents (kg Mg-1)": 7,
    "silt contents (kg Mg-1)": 8,
    "rock fraction" : 10,
    "ph" : 11,
    "SOC (kg Mg-1)": 14 
    }
    with open(soil_path, 'r') as slfl:
        lines = slfl.readlines()
        for col_n, ln in soil_prfl_vars.items():
            line_fl = lines[ln].strip().split(',')
            line_fl = [x for x in line_fl if x]
            mean = np.mean([float(i) for i in line_fl])
            df[col_n] = mean
            
        
    return df
            
            

def agrg_temp_plant_soil(temp_fl_path: str, planting_path: str, plt_fl_idx: str):
    planting =  ['planting date', 'initial planting density (m-2)', 'seeding depth (m)']
    harvesting = ['harvesting date']
    with open(planting_path, 'r') as plntfl:
        df = pd.read_csv(temp_fl_path)
        line = plntfl.readline()
        planting_data = line.strip().split(',')
        date = planting_data[0]
        planting_data =planting_data[1:]
        
        if len(planting_data) != 2:
            raise Exception("Planting data mismatch")
        
        df[planting[0] + plt_fl_idx ] = date
        df[planting[1] + plt_fl_idx] = planting_data[0]
        df[planting[2] + plt_fl_idx] = planting_data[1]
    
    
    df = handle_soil(df, '../' + cfg_files_pth + '/soil')
    df.to_csv(temp_fl_path, index=False)

### Adding all planting dates to each temperature file. There have been 7 planting dates across each year from 2001 to 2007
#### This exposes temporal variabilities between planting dates and fluxations in the target variables that the models can pick up on 

In [43]:
for i in range(0, years):
    for j in range(1, 7):
        agrg_temp_plant_soil('../datasets/' + local + '/csv_outs/' + str(growth_strt + i) + 'temp.csv',  '../'+ cfg_files_pth + '/planting' + str(j), str(j)  )